In [1]:
import bs4
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
tickers = pd.read_html('https://indiancompanies.in/listed-companies-in-nse-with-symbol/')[0]
tickers.columns = tickers.iloc[0]
tickers = tickers.iloc[1:,:]
tickers = list(tickers.SYMBOL)
print(tickers)
#print(tickers)
names = [x for x in input("enter the stock names").split()]
print(names)
fundamentals = {}

['20MICRONS', '21STCENMGM', '3IINFOTECH', '3MINDIA', '3PLAND', '5PAISA', '63MOONS', 'A2ZINFRA', 'AARTIDRUGS', 'AARTIIND', 'AARVEEDEN', 'AAVAS', 'ABAN', 'ABB', 'ABBOTINDIA', 'ABCAPITAL', 'ABFRL', 'ABMINTLTD', 'ACC', 'ACCELYA', 'ACE', 'ADANIENT', 'ADANIGAS', 'ADANIGREEN', 'ADANIPORTS', 'ADANIPOWER', 'ADANITRANS', 'ADFFOODS', 'ADHUNIK', 'ADHUNIKIND', 'ADLABS', 'ADORWELD', 'ADROITINFO', 'ADSL', 'ADVANIHOTR', 'ADVENZYMES', 'AEGISCHEM', 'AFFLE', 'AGARIND', 'AGCNET', 'AGRITECH', 'AGROPHOS', 'AHLEAST', 'AHLUCONT', 'AHLWEST', 'AIAENG', 'AIONJSW', 'AIRAN', 'AJANTPHARM', 'AJMERA', 'AKASH', 'AKSHARCHEM', 'AKSHOPTFBR', 'AKZOINDIA', 'ALANKIT', 'ALBERTDAVD', 'ALBK', 'ALCHEM', 'ALEMBICLTD', 'ALICON', 'ALKALI', 'ALKEM', 'ALKYLAMINE', 'ALLCARGO', 'ALLSEC', 'ALMONDZ', 'ALOKTEXT', 'ALPA', 'ALPHAGEO', 'ALPSINDUS', 'AMARAJABAT', 'AMBER', 'AMBIKCO', 'AMBUJACEM', 'AMDIND', 'AMJLAND', 'AMRUTANJAN', 'ANANTRAJ', 'ANDHRABANK', 'ANDHRACEMT', 'ANDHRSUGAR', 'ANIKINDS', 'ANKITMETAL', 'ANSALAPI', 'ANSALHSG', 'ANTGRAPH

In [2]:
ori = 'https://www.screener.in/'
login_route = 'login/'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36' , 'origin':'https://www.screener.in', 'referer': '/login/','referrer-policy':'same-origin'}
s=requests.session()
csrf = s.get(ori+login_route).cookies['csrftoken']
payload = {
        'username': 'jainamjain1480@gmail.com',
        'password': 'Ggaknrkr@123',
        'csrfmiddlewaretoken': csrf
    }
login_req = s.post(ori+login_route,headers=header,data=payload)
login_req.status_code

200

In [3]:
soup_beforelogin = None
soup_afterlogin =None
def scrape(name):
    req=requests.get('https://www.screener.in/company/{}/consolidated'.format(name.upper()))
    soup_beforelogin = bs4.BeautifulSoup(req.text,'html')
    def getcompany_id():
        attr =soup_beforelogin.find('main', {'class': 'flex-grow container'}).find('div',{'id':'company-info'})
        id =attr['data-warehouse-id']
        print(id)
        return id
    # ori = 'https://www.screener.in/'
    # login_route = 'login/'
    # header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36' , 'origin':'https://www.screener.in', 'referer': '/login/','referrer-policy':'same-origin'}

    # s=requests.session()
    # csrf = s.get(ori+login_route).cookies['csrftoken']

    # payload = {
    #     'username': 'jainamjain1480@gmail.com',
    #     'password': 'Ggaknrkr@123',
    #     'csrfmiddlewaretoken': csrf
    # }

    # login_req = s.post(ori+login_route,headers=header,data=payload)
    #cookies = login_req.cookies
    soup_afterlogin = BeautifulSoup(s.get(ori+'api/company/{}/quick_ratios/'.format(getcompany_id())).text,'html.parser')
    #print(soup_afterlogin)
    return soup_beforelogin, soup_afterlogin

In [4]:
def get_fund(soup_beforelogin,soup_afterlogin):
    for i in range(9):
        if i==2 or i ==8:
            continue
        else:
            name=soup_beforelogin.find_all('li',{'class': 'flex flex-space-between'})[i].find('span',{'class':'name'}).text
            num=soup_beforelogin.find_all('li',{'class': 'flex flex-space-between'})[i].find('span',{'class':'number'}).text
            if num =='':
                fundamentals[name.strip()]=np.nan
            else:
                fundamentals[name.strip()]=float(num.replace(',',''))
    for i in range(8):
        name=soup_afterlogin.find_all('li',{'class': 'flex flex-space-between'})[i].find('span',{'class':'name'}).text
        num=soup_afterlogin.find_all('li',{'class': 'flex flex-space-between'})[i].find('span',{'class':'number'}).text
        if num == '':
            fundamentals[name.strip()]=np.nan
        else:
            fundamentals[name.strip()]=float(num.replace(',',''))
    return fundamentals
#get_fund(soup_beforelogin,soup_afterlogin)

In [5]:
def pl_n_years(soup, section_id, class_name, n):
    section_html = soup.find('section',{'id': section_id})
    table_html = section_html.find('table',{'class': class_name})

    headers = []
    for header in table_html.find_all('th'):
        headers.append(  header.text or 'Type')

    table_df = pd.DataFrame(columns = headers)

    for row_ele in table_html.find_all('tr')[1:]:
            row_data = row_ele.find_all('td')
            row = [tr.text.strip() for tr in row_data]
            length = len(table_df)
            table_df.loc[length] = row 
    df = table_df[table_df['Type']=='Net Profit']
    ls = df.iloc[:,-4:-1].values
    return ls

#print(pl_n_years(soup_beforelogin,'profit-loss','data-table responsive-text-nowrap',3))

In [6]:
df =None
pl_df = None
for i in range(len(names)):
    soup_beforelogin, soup_afterlogin = scrape(names[i])
    dic = get_fund(soup_beforelogin, soup_afterlogin)
    if i==0:
        df = pd.DataFrame([dic])
    else:
        df.loc[len(df.index)] = dic
    pl = pl_n_years(soup_beforelogin,'profit-loss','data-table responsive-text-nowrap',3)
    if i == 0:
        pl_df = pd.DataFrame(pl,columns=['3 years ago','2 year ago','1 year ago'])
        #print(pl_df)
    else:
        pl = pl.flatten()
        pl_df = pl_df.append(pd.Series(pl, index=pl_df.columns[:len(pl)]), ignore_index=True)
        #print(pl_df)

print(df ,pl_df, sep = '\n')

6598877
6596470
6594812
   Market Cap  Current Price  Stock P/E  Book Value  Dividend Yield   ROCE  \
0    500537.0          561.0       12.4       342.0            1.27   4.44   
1    631385.0         1500.0       27.6       190.0            2.07  37.10   
2    296622.0         3092.0       78.8       147.0            0.62  29.70   

    ROE  Promoter holding  Debt to equity       Debt  High price  \
0  12.2              57.5           14.80  4536570.0       579.0   
1  29.0              13.1            0.08     6522.0      1954.0   
2  23.2              52.6            0.13     1866.0      3590.0   

   Intrinsic Value  Low price   EPS  Price to book value  
0            329.0      425.0  39.6                 1.64  
1            833.0     1355.0  54.4                 7.89  
2            643.0     2560.0  38.2                21.00  
  3 years ago 2 year ago 1 year ago
0      21,140     23,888     37,183
1      16,639     19,423     22,146
2       2,774      3,207      3,085


In [ ]:
soup_afterlogin.find_all('li',{'class': 'flex flex-space-between'})[0].find('span',{'class':'name'}).tex

In [ ]:
df =None
pl_df = None
for i in range(len(names)):
    soup_beforelogin, soup_afterlogin = scrape(names[i])
    dic = get_fund(soup_beforelogin, soup_afterlogin)
    if i==0:
        df = pd.DataFrame([dic])
    else:
        df.loc[len(df)] = dic
    pl = pl_n_years(soup_beforelogin,'profit-loss','data-table responsive-text-nowrap',3)
    if i == 0:
        pl_df = pd.DataFrame(pl,columns=['3 years ago','2 year ago','1 year ago'])
        #print(pl_df)
    else:
        pl = pl.flatten()
        pl_df = pl_df.append(pd.Series(pl, index=pl_df.columns[:len(pl)]), ignore_index=True)
        #print(pl_df)

display(df ,pl_df, sep = '\n')

In [7]:
!pip install git+https://github.com/alvarobartt/investpy.git@master

: 

: 

In [14]:
# import investpy 
# search_result = investpy.search_quotes(text='apple', products=['stocks'],
#                                        countries=['united states'], n_results=1)
# recent_data = search_result.retrieve_recent_data()
# historical_data = search_result.retrieve_historical_data(from_date='01/01/2019', to_date='01/01/2020')
# information = search_result.retrieve_information()
# default_currency = search_result.retrieve_currency()
# technical_indicators = search_result.retrieve_technical_indicators(interval='daily')

ConnectionError: ERR#0015: error 403, try again later.

In [ ]:
'''
web scrape to login into a website
url = 'https://www.screener.in/login'
Head = {'referer' : 'https://www.screener.in/login/?','referrer-policy':'same-origin'}
payload = {
    'username': 'jainamjain1480@gmail.com',
    'password': 'Ggaknrkr@123'
}
with requests.session() as s:
    r = s.post(url,data = payload,headers=Head)
    print(r.status_code)'''

In [4]:
a =['SBIN','RENUKA','INFY']
b= 'SBIN'
if b in a:
    print('yes')

yes
